# Data Understanding

In [2]:
from pyspark.sql import SparkSession, functions as F

import findspark

findspark.init("C:\Program Files\Spark\spark-3.3.1-bin-hadoop3")

In [3]:
spark = (SparkSession.builder
            .appName("Data Understanding")
            .master("local[2]")
            .config("spark.sql.adaptive", True)
            .getOrCreate())

In [4]:
iot_schema = "row_id int, " \
    "ts double, " \
    "device string, " \
    "co float, " \
    "humidity float, " \
    "light boolean, " \
    "lpg float, " \
    "motion boolean, " \
    "smoke float, " \
    "temp float, " \
    "time timestamp"

In [5]:
path = "file:///Users/talha/OneDrive/Masaüstü/Talha Nebi Kumru/Data Enginnering/Miuul/RealTimeDPWithSpark/Windowed_Aggregation_Spark_Streaming/datasets/output"

df = (spark.read
         .format("csv")
         .schema(iot_schema)
         .option("header", True)
         .load(path))

In [6]:
df.limit(5).toPandas()

C:\Users\talha\anaconda3\lib\site-packages\pyspark\sql\pandas\conversion.py:194: FutureWarning: Passing unit-less datetime64 dtype to .astype is deprecated and will raise in a future version. Pass 'datetime64[ns]' instead
  series = series.astype(t, copy=False)


,row_id,ts,device,co,humidity,light,lpg,motion,smoke,temp,time
0,1671,1.594515e+09,00:0f:00:70:91:0a,0.002613,75.300003,False,0.004815,False,0.012445,19.799999,2023-02-15 22:51:54.981056
1,1672,1.594515e+09,1c:bf:ce:15:ec:4d,0.004215,77.800003,True,0.006805,False,0.018010,26.900000,2023-02-15 22:51:55.492817
2,1673,1.594515e+09,b8:27:eb:bf:9d:51,0.004992,50.599998,False,0.007691,False,0.020525,22.600000,2023-02-15 22:51:56.005189
3,1674,1.594515e+09,00:0f:00:70:91:0a,0.002584,75.300003,False,0.004777,False,0.012340,19.700001,2023-02-15 22:51:56.520375
4,1675,1.594515e+09,b8:27:eb:bf:9d:51,0.005029,50.500000,False,0.007733,False,0.020645,22.600000,2023-02-15 22:51:57.034116


In [7]:
windowed_file = df.groupBy(F.window(F.col("time"), "10 minutes", "5 minutes"),
                             F.col("device")).agg(
                                     F.count("device").alias("Count"),
                                     F.avg("humidity").alias("ten_min_avg(humidity)"),
                                     F.avg("co").alias("ten_min_avg(humidity)"))

In [8]:
windowed_file.limit(5).toPandas()

,window,device,Count,ten_min_avg(humidity),ten_min_avg(humidity)
0,"(2023-02-15 23:15:00, 2023-02-15 23:25:00)",1c:bf:ce:15:ec:4d,140,76.939287,0.004196
1,"(2023-02-15 23:00:00, 2023-02-15 23:10:00)",1c:bf:ce:15:ec:4d,306,76.932680,0.004131
2,"(2023-02-15 23:10:00, 2023-02-15 23:20:00)",1c:bf:ce:15:ec:4d,293,76.974404,0.004155
3,"(2023-02-15 23:05:00, 2023-02-15 23:15:00)",00:0f:00:70:91:0a,268,75.729851,0.002366
4,"(2023-02-15 23:00:00, 2023-02-15 23:10:00)",00:0f:00:70:91:0a,280,75.939643,0.002386
